## Batch processing with Argo Worfklows

In this notebook we will dive into how you can run batch processing with Argo Workflows and Seldon Core.

Dependencies:

* Seldon core installed as per the docs with an ingress
* Argo Workfklows installed in cluster (and argo CLI for commands)


## Argo Workflows Example

Let's try an argo workflows example to see intuitively how it works. 

In this case we will trigger a workflow with 3 steps (first one will execute and the other two jobs are dependent on that).

The example below will basically run a workflow in the following order:

![](assets/argo-example.jpg)

In [1]:
mkdir -p assets

In [821]:
%%writefile assets/argo-example.yaml
---
apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  name: argo-basic-example
spec:
  entrypoint: hello-hello-hello
  # This spec contains two templates: hello-hello-hello and whalesay
  templates:
  - name: hello-hello-hello
    # Instead of just running a container
    # This template has a sequence of steps
    steps:
    - - name: hello1            # hello1 is run before the following steps
        template: whalesay
        arguments:
          parameters:
          - name: message
            value: "hello1"
    - - name: hello2a           # double dash => run after previous step
        template: whalesay
        arguments:
          parameters:
          - name: message
            value: "hello2a"
      - name: hello2b           # single dash => run in parallel with previous step
        template: whalesay
        arguments:
          parameters:
          - name: message
            value: "hello2b"
  # This is the same template as from the previous example
  - name: whalesay
    inputs:
      parameters:
      - name: message
    container:
      image: docker/whalesay
      command: [cowsay]
      args: ["{{inputs.parameters.message}}"]
        

Overwriting assets/argo-example.yaml


In [822]:
!argo submit assets/argo-example.yaml

Name:                argo-basic-example
Namespace:           default
ServiceAccount:      default
Status:              Pending
Created:             Sat May 02 12:15:51 +0100 (now)


In [823]:
!argo list

NAME                 STATUS    AGE   DURATION   PRIORITY
argo-basic-example   Running   2s    2s         0


In [863]:
!argo get argo-basic-example

Name:                argo-basic-example
Namespace:           default
ServiceAccount:      default
Status:              Succeeded
Created:             Sat May 02 12:15:51 +0100 (47 seconds ago)
Started:             Sat May 02 12:15:51 +0100 (47 seconds ago)
Finished:            Sat May 02 12:16:38 +0100 (now)
Duration:            47 seconds

STEP                                       PODNAME                        DURATION  MESSAGE
 ✔ argo-basic-example (hello-hello-hello)                                           
 ├---✔ hello1 (whalesay)                   argo-basic-example-3663002391  39s       
 └-·-✔ hello2a (whalesay)                  argo-basic-example-3531035296  3s        
   └-✔ hello2b (whalesay)                  argo-basic-example-3581368153  5s        


In [864]:
!argo logs -w argo-basic-example

hello1:	 ________ 
hello1:	< hello1 >
hello1:	 -------- 
hello1:	    \
hello1:	     \
hello1:	      \     
hello1:	                    ##        .            
hello1:	              ## ## ##       ==            
hello1:	           ## ## ## ##      ===            
hello1:	       /""""""""""""""""___/ ===        
hello1:	  ~~~ {~~ ~~~~ ~~~ ~~~~ ~~ ~ /  ===- ~~~   
hello1:	       \______ o          __/            
hello1:	        \    \        __/             
hello1:	          \____\______/   
hello2a:	 _________ 
hello2a:	< hello2a >
hello2a:	 --------- 
hello2a:	    \
hello2a:	     \
hello2a:	      \     
hello2a:	                    ##        .            
hello2a:	              ## ## ##       ==            
hello2a:	           ## ## ## ##      ===            
hello2a:	       /""""""""""""""""___/ ===        
hello2a:	  ~~~ {~~ ~~~~ ~~~ ~~~~ ~~ ~ /  ===- ~~~   
hello2a:	       \______ o          __/            
hello2a:	        \    \        __/             
hello2a:	          \____\__

In [865]:
!argo delete argo-basic-example

Workflow 'argo-basic-example' deleted


## Seldon Core Batch 
Now we can leverage this functionality by using seldon core batch.

The structure of this job will be the following:

![](assets/seldon-batch-simple.jpg)

THe file below denotes the structure of the three steps in this workflow:

In [798]:
%%writefile assets/seldon-batch.yaml
---
apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  name: seldon-batch-simple
spec:
  entrypoint: seldon-batch-process
  templates:
  - name: seldon-batch-process
    steps:
    - - name: create-seldon-resource            
        template: create-seldon-resource-template
    - - name: wait-seldon-resource
        template: wait-seldon-resource-template
    - - name: process-batch-inputs
        template: process-batch-inputs-template
            
  - name: create-seldon-resource-template
    resource:
      action: create
      manifest: |
        apiVersion: machinelearning.seldon.io/v1
        kind: SeldonDeployment
        metadata:
          name: "{{workflow.uid}}"
          ownerReferences:
          - apiVersion: argoproj.io/v1alpha1
            blockOwnerDeletion: true
            kind: Workflow
            name: "{{workflow.name}}"
            uid: "{{workflow.uid}}"
        spec:
          name: "{{workflow.uid}}"
          predictors:
            - graph:
                children: []
                implementation: SKLEARN_SERVER
                modelUri: gs://seldon-models/sklearn/iris
                name: classifier
              name: default
              replicas: 1
                
  - name: wait-seldon-resource-template
    script:
      image: seldonio/core-builder:0.14
      command: [bash]
      source: |
        sleep 5
        kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id="{{workflow.uid}}" -o jsonpath='{.items[0].metadata.name}')
        
  - name: process-batch-inputs-template
    script:
      image: seldonio/seldon-core-s2i-python3:1.1.1-SNAPSHOT
      command: [python]
      source: |
        from seldon_core.seldon_client import SeldonClient
        import numpy as np
        import time
        sc = SeldonClient(
            gateway_endpoint="istio-ingressgateway.istio-system.svc.cluster.local",
            deployment_name="{{workflow.uid}}",
            namespace="default")
        for i in range(10):
            data = np.array([[i, i, i, i]])
            output = sc.predict(data=data)
            print(output.response)
            

Overwriting assets/seldon-batch.yaml


In [799]:
!argo submit assets/seldon-batch.yaml

Name:                seldon-batch-simple
Namespace:           default
ServiceAccount:      default
Status:              Pending
Created:             Sat May 02 12:14:04 +0100 (now)


In [801]:
!argo list

NAME                  STATUS    AGE   DURATION   PRIORITY
seldon-batch-simple   Running   8s    8s         0


In [802]:
!argo get seldon-batch-simple

Name:                seldon-batch-simple
Namespace:           default
ServiceAccount:      default
Status:              Running
Created:             Sat May 02 12:14:04 +0100 (9 seconds ago)
Started:             Sat May 02 12:14:04 +0100 (9 seconds ago)
Duration:            9 seconds

STEP                                                             PODNAME                         DURATION  MESSAGE
 ● seldon-batch-simple (seldon-batch-process)                                                              
 ├---✔ create-seldon-resource (create-seldon-resource-template)  seldon-batch-simple-3724798319  2s        
 └---● wait-seldon-resource (wait-seldon-resource-template)      seldon-batch-simple-4145437349  7s        


In [815]:
!argo logs -w seldon-batch-simple

create-seldon-resource:	time="2020-05-02T11:14:05Z" level=info msg="Starting Workflow Executor" version=vv2.7.4+50b209c.dirty
create-seldon-resource:	time="2020-05-02T11:14:05Z" level=info msg="Creating a docker executor"
create-seldon-resource:	time="2020-05-02T11:14:05Z" level=info msg="Executor (version: vv2.7.4+50b209c.dirty, build_date: 2020-04-16T16:37:57Z) initialized (pod: default/seldon-batch-simple-3724798319) with template:\n{\"name\":\"create-seldon-resource-template\",\"arguments\":{},\"inputs\":{},\"outputs\":{},\"metadata\":{},\"resource\":{\"action\":\"create\",\"manifest\":\"apiVersion: machinelearning.seldon.io/v1\\nkind: SeldonDeployment\\nmetadata:\\n  name: \\\"b754cbcc-e2f2-49ee-978b-643699dee396\\\"\\n  ownerReferences:\\n  - apiVersion: argoproj.io/v1alpha1\\n    blockOwnerDeletion: true\\n    kind: Workflow\\n    name: \\\"seldon-batch-simple\\\"\\n    uid: \\\"b754cbcc-e2f2-49ee-978b-643699dee396\\\"\\nspec:\\n  name: \\\"b754cbcc-e2f2-49ee-978b-643699dee396\\

In [819]:
outputs = !(argo logs -w seldon-batch-simple --no-color | grep "process-batch-inputs" | cut -c 23-)
for o in outputs:
    print(o)

{'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.3664487684438811, 0.48528762951761806, 0.14826360203850078]}}, 'meta': {}}
{'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.2075509473561692, 0.2443463805811625, 0.5481026720626684]}}, 'meta': {}}
{'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.06995304386311439, 0.04864300564562103, 0.8814039504912645]}}, 'meta': {}}
{'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.01859472366015777, 0.006956450489196832, 0.9744488258506454]}}, 'meta': {}}
{'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.004653433216061866, 0.0009398331072469446, 0.9944067336766912]}}, 'meta': {}}
{'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.0011463235173706913, 0.0001256712307515923, 0.9987280052518777]}}, 'meta': {}}
{'data': {'names': ['t:0', 't:1', 't:2'], 'ten

In [820]:
!argo delete seldon-batch-simple

Workflow 'seldon-batch-simple' deleted


## Seldon Core Batch with Object Store

In some cases we may want to read the data from an object source.

In this case we will show how you can read from an object store, in this case minio.

The workflow will look as follows:

![](assets/seldon-batch-store.jpg)

For this we will assume you have installed the Minio (mc) CLI - we will use a Minio client in the cluster but you can use another object store provider like S3, Google Cloud, Azure, etc.

### Set up Minio in your cluster

In [481]:
%%bash 
helm install minio stable/minio \
    --set accessKey=minioadmin \
    --set secretKey=minioadmin \
    --set image.tag=RELEASE.2020-04-15T19-42-18Z

NAME: minio
LAST DEPLOYED: Thu Apr 30 10:57:00 2020
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Minio can be accessed via port 9000 on the following DNS name from within your cluster:
minio.default.svc.cluster.local

To access Minio from localhost, run the below commands:

  1. export POD_NAME=$(kubectl get pods --namespace default -l "release=minio" -o jsonpath="{.items[0].metadata.name}")

  2. kubectl port-forward $POD_NAME 9000 --namespace default

Read more about port forwarding here: http://kubernetes.io/docs/user-guide/kubectl/kubectl_port-forward/

You can now access Minio server on http://localhost:9000. Follow the below steps to connect to Minio server with mc client:

  1. Download the Minio mc client - https://docs.minio.io/docs/minio-client-quickstart-guide

  2. mc config host add minio-local http://localhost:9000 minioadmin minioadmin S3v4

  3. mc ls minio-local

Alternately, you can use your browser or the Minio SDK to access the server - ht

### Forward the Minio port so you can access it

You can do this by runnning the following command in your terminal:
```
kubectl port-forward svc/minio 9000:9000
    ```
    
### Configure local minio client

In [483]:
!mc config host add minio-local http://localhost:9000 minioadmin minioadmin

Added `minio-local` successfully.


### Configure bucket for Argo Artefact passing

In [76]:
%%writefile assets/argo-config.yaml
---
apiVersion: v1
kind: ConfigMap
metadata:
  name: workflow-controller-configmap
data:
  config: |
    artifactRepository:
      s3:
        bucket: argo-artifacts
        endpoint: minio.default.svc.cluster.local:9000
        insecure: true
        accessKeySecret:
          name: minio
          key: accesskey
        secretKeySecret:
          name: minio
          key: secretkey

Overwriting assets/argo-config.yaml


### Make sure the configmap is in the same location as your argo controller

In [108]:
!kubectl apply -n argo -f assets/argo-config.yaml

configmap/workflow-controller-configmap unchanged


### Create the bucket for artifact passing

In [572]:
!mc mb minio-local/argo-artifacts

Bucket created successfully `minio-local/argo-artifacts`.


### Create some input for our model

We will create a file that will contain the inputs that will be sent to our model

In [1]:
with open("assets/input-data.txt", "w") as f:
    for i in range(10000):
        f.write('{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "' + str(i) + '"}}\n')

### Check the contents of the file

In [2]:
!wc -l assets/input-data.txt
!head assets/input-data.txt

10000 assets/input-data.txt
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "0"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "1"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "2"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "3"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "4"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "5"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "6"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "7"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "8"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "9"}}


### Upload the file to our minio

In [3]:
!mc mb minio-local/data
!mc cp assets/input-data.txt minio-local/data/

mc: <ERROR> Unable to make bucket `minio-local/data`. Your previous request to create the named bucket succeeded and you already own it.
...-data.txt:  614.15 KiB / 614.15 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 5.67 MiB/s 0s

### Create Argo Workflow

In order to create our argo workflow we have made it simple so you can leverage the power of the helm charts.

Before we dive into the contents of the full helm chart, let's first give it a try with some of the settings.

We will run a batch job that will set up a Seldon Deployment with 10 replicas and 100 batch client workers to send requests.

In [48]:
!helm template seldon-batch-workflow helm-charts/seldon-batch-workflow/ \
    --set workflow.name=seldon-batch-process \
    --set seldonDeployment.name=sklearn \
    --set seldonDeployment.replicas=10 \
    --set batchWorker.workers=100 \
    | argo submit -

Name:                seldon-batch-process
Namespace:           default
ServiceAccount:      default
Status:              Pending
Created:             Wed Jun 03 18:46:13 +0100 (now)


In [49]:
!argo list

NAME                   STATUS    AGE   DURATION   PRIORITY
seldon-batch-process   Running   1m    1m         0


In [55]:
!argo get seldon-batch-process

Name:                seldon-batch-process
Namespace:           default
ServiceAccount:      default
Status:              Running
Created:             Wed Jun 03 18:46:13 +0100 (27 minutes ago)
Started:             Wed Jun 03 18:46:13 +0100 (27 minutes ago)
Duration:            27 minutes 59 seconds

STEP                                                             PODNAME                          DURATION  MESSAGE
 ● seldon-batch-process (seldon-batch-process)                                                              
 ├---✔ create-seldon-resource (create-seldon-resource-template)  seldon-batch-process-3626514072  3s        
 ├---✔ wait-seldon-resource (wait-seldon-resource-template)      seldon-batch-process-2052519094  30s       
 ├---✔ download-object-store (download-object-store-template)    seldon-batch-process-1257652469  4s        
 └---● process-batch-inputs (process-batch-inputs-template)      seldon-batch-process-2033515954  27m       


In [56]:
!argo logs -w seldon-batch-process 

create-seldon-resource:	time="2020-06-03T17:46:15Z" level=info msg="Starting Workflow Executor" version=v2.8.0-rc4+8f69617.dirty
create-seldon-resource:	time="2020-06-03T17:46:15Z" level=info msg="Creating a docker executor"
create-seldon-resource:	time="2020-06-03T17:46:15Z" level=info msg="Executor (version: v2.8.0-rc4+8f69617.dirty, build_date: 2020-05-12T15:17:15Z) initialized (pod: default/seldon-batch-process-3626514072) with template:\n{\"name\":\"create-seldon-resource-template\",\"arguments\":{},\"inputs\":{},\"outputs\":{},\"metadata\":{},\"resource\":{\"action\":\"create\",\"manifest\":\"apiVersion: machinelearning.seldon.io/v1\\nkind: SeldonDeployment\\nmetadata:\\n  name: \\\"sklearn\\\"\\n  namespace: default\\n  ownerReferences:\\n  - apiVersion: argoproj.io/v1alpha1\\n    blockOwnerDeletion: true\\n    kind: Workflow\\n    name: \\\"seldon-batch-process\\\"\\n    uid: \\\"b855a708-e036-4bcf-9ec8-354916f8fd1c\\\"\\nspec:\\n  name: \\\"sklearn\\\"\\n  predictors:\\n    - 

process-batch-inputs:	  File "/opt/conda/lib/python3.7/site-packages/urllib3/connection.py", line 159, in _new_conn
process-batch-inputs:	    (self._dns_host, self.port), self.timeout, **extra_kw)
process-batch-inputs:	  File "/opt/conda/lib/python3.7/site-packages/urllib3/util/connection.py", line 80, in create_connection
process-batch-inputs:	    raise err
process-batch-inputs:	  File "/opt/conda/lib/python3.7/site-packages/urllib3/util/connection.py", line 70, in create_connection
process-batch-inputs:	    sock.connect(sa)
process-batch-inputs:	ConnectionRefusedError: [Errno 111] Connection refused
process-batch-inputs:	During handling of the above exception, another exception occurred:
process-batch-inputs:	Traceback (most recent call last):
process-batch-inputs:	  File "/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py", line 600, in urlopen
process-batch-inputs:	    chunked=chunked)
process-batch-inputs:	  File "/opt/conda/lib/python3.7/site-packages/urllib3/connec

process-batch-inputs:	urllib3.exceptions.NewConnectionError: <urllib3.connection.HTTPConnection object at 0x7f00fc08d310>: Failed to establish a new connection: [Errno 111] Connection refused
process-batch-inputs:	During handling of the above exception, another exception occurred:
process-batch-inputs:	Traceback (most recent call last):
process-batch-inputs:	  File "/opt/conda/lib/python3.7/site-packages/requests/adapters.py", line 449, in send
process-batch-inputs:	    timeout=timeout
process-batch-inputs:	  File "/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py", line 638, in urlopen
process-batch-inputs:	    _stacktrace=sys.exc_info()[2])
process-batch-inputs:	  File "/opt/conda/lib/python3.7/site-packages/urllib3/util/retry.py", line 399, in increment
process-batch-inputs:	    raise MaxRetryError(_pool, url, error or ResponseError(cause))
process-batch-inputs:	urllib3.exceptions.MaxRetryError: HTTPConnectionPool(host='istio-ingressgateway.istio-system.svc.cluster.lo

process-batch-inputs:	    _stacktrace=sys.exc_info()[2])
process-batch-inputs:	  File "/opt/conda/lib/python3.7/site-packages/urllib3/util/retry.py", line 399, in increment
process-batch-inputs:	    raise MaxRetryError(_pool, url, error or ResponseError(cause))
process-batch-inputs:	urllib3.exceptions.MaxRetryError: HTTPConnectionPool(host='istio-ingressgateway.istio-system.svc.cluster.local', port=80): Max retries exceeded with url: /seldon/default/sklearn/api/v1.0/predictions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f00fc3d7610>: Failed to establish a new connection: [Errno 111] Connection refused'))
process-batch-inputs:	During handling of the above exception, another exception occurred:
process-batch-inputs:	Traceback (most recent call last):
process-batch-inputs:	  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
process-batch-inputs:	    self.run()
process-batch-inputs:	  File "/opt/conda/lib/python3.7/threading.py"

process-batch-inputs:	requests.exceptions.ConnectionError: HTTPConnectionPool(host='istio-ingressgateway.istio-system.svc.cluster.local', port=80): Max retries exceeded with url: /seldon/default/sklearn/api/v1.0/predictions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f00fc131b50>: Failed to establish a new connection: [Errno 111] Connection refused'))
process-batch-inputs:	Exception in thread Thread-88:
process-batch-inputs:	Traceback (most recent call last):
process-batch-inputs:	  File "/opt/conda/lib/python3.7/site-packages/urllib3/connection.py", line 159, in _new_conn
process-batch-inputs:	    (self._dns_host, self.port), self.timeout, **extra_kw)
process-batch-inputs:	  File "/opt/conda/lib/python3.7/site-packages/urllib3/util/connection.py", line 80, in create_connection
process-batch-inputs:	    raise err
process-batch-inputs:	  File "/opt/conda/lib/python3.7/site-packages/urllib3/util/connection.py", line 70, in create_connection
process-batc

## Check output in object store

We can now visualise the output that we obtained in the object store.

First we can check that the file is present:

In [775]:
import json
wf_arr = !argo get seldon-batch-process -o json
wf = json.loads("".join(wf_arr))
WF_ID = wf["metadata"]["uid"]
print(f"Workflow ID is {WF_ID}")

Workflow ID is 72ad4430-c70a-4976-8043-2ba6c2f56772


In [776]:
!mc ls minio-local/data/output-data-"$WF_ID".txt

[2020-05-02 12:07:51 BST]  1.3MiB output-data-72ad4430-c70a-4976-8043-2ba6c2f56772.txt


Now we can output the contents of the file created using the `mc head` command.

In [777]:
!mc cp minio-local/data/output-data-"$WF_ID".txt assets/output-data.txt
!head assets/output-data.txt

...56772.txt:  1.31 MiB / 1.31 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 23.35 MiB/s 0s{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"1"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"5"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"0"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"3"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"7"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"2"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478

In [47]:
!argo delete seldon-batch-process

Workflow 'seldon-batch-process' deleted
